IMPORTAR RECURSOS

In [14]:
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from functions import *
from getTweets import *
from processTopic import *
import pandas as pd
import numpy as np
from numpy import loadtxt
import operator

# **Obtener Tweets**

In [53]:
# Obtener trends
trends = getTrend()
print(trends)

[✓] Trends successfully extracted.
['#ClimateAction', '#TheDay_sevENstars_Connected', '#TheLateShowMeMusic', '#nochedeconfesiones', '#엔하이픈_7개_별들이_연결된_날', 'AGUST D RECORD SETTER', 'ATEEZ STAY COVER', 'Aldair', 'Alianza', 'Arley', 'BLACKPINK ONU', 'Barcos', 'Benítez', 'Bryan', 'Burga', 'Bustos', 'COLBERT', 'Calcaterra', 'Congreso', 'Corzo', 'Cusco FC', 'Damir Dzumhur', 'ENHYPEN 1ST ANNIVERSARY', 'España', 'Farfán', 'Fase 2', 'Guillén', 'Gustabo', 'Irven Ávila', 'La Resistencia', 'Liza', 'Lora', 'Melgar', 'Merino', 'Mora', 'Moyano', 'Otis', 'Pirata', 'RECORD BREAKING FILTER', 'Rengifo', 'Riquelme', 'Sporting Cristal', 'THE ENHYPEN DIMENSION OPENS', 'Tubino', 'danceracha x etro', 'leao', 'mosquera', 'pa que guaye tonight', 'raziel garcía', 'taylor']


In [54]:
# Obtener tweets
get_tweets_tweepy("taylor")

[✓] 500 tweets received about taylor


# **Crear Noticia**

## Leer trend en json

In [93]:
trend_path = 'tweets\spiderman.json'
data = pd.read_json(trend_path, lines=True)
data.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,language
0,1438923095935488000,"Star Wars Episodio I, fue el primer trailer qu...",GabyMeza8,228647,31,336,es
1,1436286323460083712,En 2002 se estrenó la primera película de la s...,manzottipablo,38065,320,1606,es
2,1436069507148177408,OFICIAL: #Sony confirma que #InsomniacGames es...,GabyMeza8,228647,213,2784,es
3,1439007421670768640,@AntipipasSX Las sagas mas icónicas? que yo se...,Hyacintho_Draco,333,0,0,es
4,1439007379195113472,@Nuezmacadamia Jo el mejor spiderman es Peter ...,Rafamusic_Vzla,294,0,0,es


## Limpieza de datos

In [94]:
data_clean = clean_data(trend_path)
data_clean.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,language
0,1438923095935488000,star wars episodio i fue el primer trailer que...,GabyMeza8,228647,31,336,es
1,1436286323460083712,en se estreno la primera pelicula de la saga...,manzottipablo,38065,320,1606,es
2,1436069507148177408,oficial sony confirma que insomniacgames esta ...,GabyMeza8,228647,213,2784,es
3,1439007421670768640,antipipassx las sagas mas iconicas que yo sep...,Hyacintho_Draco,333,0,0,es
4,1439007379195113472,nuezmacadamia jo el mejor spiderman es peter ...,Rafamusic_Vzla,294,0,0,es


## Palabras de mayor importancia - Count Vectorizer

In [95]:
# Definir stopwords
stop_word_es = get_stop_words('spanish')
def find_top_words(data):
    # contabilizar frecuencias mediante CV
    cv = CountVectorizer(stop_words= stop_word_es)
    cv_matrix = cv.fit_transform(data['text'])
    count_words = np.sum(cv_matrix.toarray(), axis=0)
    words_cv =  {key:count_words[cv.vocabulary_[key]] for key in cv.vocabulary_}
    top_words_cv = sorted(words_cv.items(), key=operator.itemgetter(1), reverse=True)

    # contabilizar frecuencia mediante IDF
    tfidf = TfidfTransformer()
    tfidf_matrix = tfidf.fit_transform(cv_matrix)
    words_idf =  dict(zip(cv.get_feature_names(), tfidf.idf_))
    top_words_idf =  sorted(words_idf.items(), key=operator.itemgetter(1), reverse=True)
    return top_words_cv, top_words_idf

In [96]:
cv, idf = find_top_words(data_clean)

## Clasidicacion mediante glosarios

In [97]:
# Cargar glosarios
glossaries = {}
with os.scandir('glossaries') as gloss_scanIte:
    for dir in gloss_scanIte:
        tmp = open(dir, 'r').readlines()
        lines = [ without_accents(word.rsplit('\n')[0]) for word in tmp]
        name = dir.path.split('\\')[-1].split('.')[0]
        glossaries[name] = lines

# Identificar categoria del trend 
points = {}
for key in glossaries:
    count = 0
    for word, score in cv:
        if word in glossaries[key]: 
            count += 1 * score
    points[key] = count
points

{'economia': 0, 'entretenimiento': 1, 'politica': 0, 'salud': 0}

## Identificacion de titular

In [113]:
data_top_by_favorite = data.sort_values('favorite_count', ascending=False)
data_top_by_favorite.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,language
2,1436069507148177408,OFICIAL: #Sony confirma que #InsomniacGames es...,GabyMeza8,228647,213,2784,es
1,1436286323460083712,En 2002 se estrenó la primera película de la s...,manzottipablo,38065,320,1606,es
0,1438923095935488000,"Star Wars Episodio I, fue el primer trailer qu...",GabyMeza8,228647,31,336,es
3,1439007421670768640,@AntipipasSX Las sagas mas icónicas? que yo se...,Hyacintho_Draco,333,0,0,es
4,1439007379195113472,@Nuezmacadamia Jo el mejor spiderman es Peter ...,Rafamusic_Vzla,294,0,0,es


In [112]:
data_top_by_user_followers = data.sort_values('user_followers', ascending=False)
data_top_by_user_followers.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,language
0,1438923095935488000,"Star Wars Episodio I, fue el primer trailer qu...",GabyMeza8,228647,31,336,es
2,1436069507148177408,OFICIAL: #Sony confirma que #InsomniacGames es...,GabyMeza8,228647,213,2784,es
1,1436286323460083712,En 2002 se estrenó la primera película de la s...,manzottipablo,38065,320,1606,es
8,1439006819884707840,#Marvel’s #SpiderMan: Miles Morales Ultimate E...,Playthusiast,10429,0,0,es
6,1439006987233202176,https://t.co/OFhVVCzkva Spiderman de Maguire e...,atletista1989,5346,0,0,es


In [117]:
data_top_by_retweet_count = data.sort_values('retweet_count', ascending=False)
data_top_by_retweet_count.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,language
1,1436286323460083712,En 2002 se estrenó la primera película de la s...,manzottipablo,38065,320,1606,es
2,1436069507148177408,OFICIAL: #Sony confirma que #InsomniacGames es...,GabyMeza8,228647,213,2784,es
0,1438923095935488000,"Star Wars Episodio I, fue el primer trailer qu...",GabyMeza8,228647,31,336,es
3,1439007421670768640,@AntipipasSX Las sagas mas icónicas? que yo se...,Hyacintho_Draco,333,0,0,es
4,1439007379195113472,@Nuezmacadamia Jo el mejor spiderman es Peter ...,Rafamusic_Vzla,294,0,0,es


In [126]:
data_score = data.copy()
data_score['score'] = data_score['user_followers'] * data_score['retweet_count'] * data_score['favorite_count']
data_score_top_by_score = data_score.sort_values('score', ascending=False)
data_score_top_by_score.head()

,id,text,screen_name,user_followers,retweet_count,favorite_count,language,score
2,1436069507148177408,OFICIAL: #Sony confirma que #InsomniacGames es...,GabyMeza8,228647,213,2784,es,135585841824
1,1436286323460083712,En 2002 se estrenó la primera película de la s...,manzottipablo,38065,320,1606,es,19562364800
0,1438923095935488000,"Star Wars Episodio I, fue el primer trailer qu...",GabyMeza8,228647,31,336,es,2381587152
3,1439007421670768640,@AntipipasSX Las sagas mas icónicas? que yo se...,Hyacintho_Draco,333,0,0,es,0
4,1439007379195113472,@Nuezmacadamia Jo el mejor spiderman es Peter ...,Rafamusic_Vzla,294,0,0,es,0
